In [98]:
from dotenv import load_dotenv
import os
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from operator import add as add_messages
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.tools import tool

In [99]:
load_dotenv()

True

In [100]:
llm = ChatOpenAI(
    model="gpt-4o", temperature = 0) # I want to minimize hallucination - temperature = 0 makes the model output more deterministic 

# Our Embedding Model - has to also be compatible with the LLM
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

pdf_path = "Stock_Market_Performance_2024.pdf"

In [101]:
# Safety measure I have put for debugging purposes :)
if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"PDF file not found: {pdf_path}")

pdf_loader = PyPDFLoader(pdf_path) # This loads the PDF

# Checks if the PDF is there
try:
    pages = pdf_loader.load()
    print(f"PDF has been loaded and has {len(pages)} pages")
except Exception as e:
    print(f"Error loading PDF: {e}")
    raise

# Chunking Process
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)


PDF has been loaded and has 9 pages


In [105]:
# ULTIMATE FIX - Create vector store with explicit client settings
from pathlib import Path
import shutil
import chromadb
from chromadb.config import Settings

# Split the documents first
pages_split = text_splitter.split_documents(pages)

# Use current working directory which is guaranteed to be writable
vector_store_dir = Path.cwd() / "vector_store_ultimate_fix"
collection_name = "stock_market"

# Clean up any existing database that might be corrupted
if vector_store_dir.exists():
    shutil.rmtree(vector_store_dir, ignore_errors=True)

vector_store_dir.mkdir(exist_ok=True)

print(f"✨ Vector store directory: {vector_store_dir}")
print(f"✅ Directory is writable: {os.access(vector_store_dir, os.W_OK)}")
print(f"📄 Processing {len(pages_split)} document chunks...")

try:
    # Create ChromaDB client with explicit settings to avoid tenant issues
    client_settings = Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=str(vector_store_dir),
        anonymized_telemetry=False
    )
    
    # Create the vector store with explicit client settings
    vectorstore = Chroma.from_documents(
        documents=pages_split,
        embedding=embeddings,
        persist_directory=str(vector_store_dir),
        collection_name=collection_name,
        client_settings=client_settings
    )
    
    print(f"🎉 Successfully created ChromaDB vector store!")
    print(f"📊 Stored {len(pages_split)} document chunks")
    
    # Test a simple query to verify it works
    test_query = "What is the stock market performance?"
    results = vectorstore.similarity_search(test_query, k=2)
    print(f"✅ Vector store is working! Found {len(results)} relevant documents for test query.")
    
except Exception as e:
    print(f"❌ Error setting up ChromaDB: {str(e)}")
    print(f"📍 Current working directory: {Path.cwd()}")
    print(f"📁 Trying to write to: {vector_store_dir}")
    print(f"📂 Path exists: {vector_store_dir.exists()}")
    print(f"✏️  Path is writable: {os.access(vector_store_dir, os.W_OK)}")
    
    # If still failing, try alternative approach
    print("\n🔄 Trying alternative approach with in-memory database...")
    try:
        vectorstore = Chroma.from_documents(
            documents=pages_split,
            embedding=embeddings,
            collection_name=collection_name + "_memory"
            # No persist_directory = in-memory only
        )
        print("✅ In-memory vector store created successfully!")
    except Exception as e2:
        print(f"❌ Alternative approach also failed: {str(e2)}")
        raise


✨ Vector store directory: /Users/ashwanthfernando/code/ml-notebooks/agentic-ai/vector_store_ultimate_fix
✅ Directory is writable: True
📄 Processing 24 document chunks...
❌ Error setting up ChromaDB: You are using a deprecated configuration of Chroma.

If you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/deployment/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/deployment/migration for more information or join our discord at https://discord.gg/MMeYNTmh3x for help!
📍 Current working directory: /Users/ashwa

In [106]:
# Now we create our retriever 
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5} # K is the amount of chunks to return
)

In [108]:
@tool
def retriever_tool(query: str) -> str:
    """
    This tool searches and returns the information from the Stock Market Performance 2024 document.
    """

    docs = retriever.invoke(query)

    if not docs:
        return "I found no relevant information in the Stock Market Performance 2024 document."
    
    results = []
    for i, doc in enumerate(docs):
        results.append(f"Document {i+1}:\n{doc.page_content}")
    
    return "\n\n".join(results)

tools = [retriever_tool]

llm = llm.bind_tools(tools)

In [109]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [86]:
def should_continue(state: AgentState):
    """Check if the last message contains tool calls."""
    result = state['messages'][-1]
    return hasattr(result, 'tool_calls') and len(result.tool_calls) > 0

In [110]:
system_prompt = """
You are an intelligent AI assistant who answers questions about Stock Market Performance in 2024 based on the PDF document loaded into your knowledge base.
Use the retriever tool available to answer questions about the stock market performance data. You can make multiple calls if needed.
If you need to look up some information before asking a follow up question, you are allowed to do that!
Please always cite the specific parts of the documents you use in your answers.
"""

In [111]:
tools_dict = {our_tool.name: our_tool for our_tool in tools} # Creating a dictionary of our tools

In [112]:
# LLM Agent
def call_llm(state: AgentState) -> AgentState:
    """Function to call the LLM with the current state."""
    messages = list(state['messages'])
    messages = [SystemMessage(content=system_prompt)] + messages
    message = llm.invoke(messages)
    return {'messages': [message]}


In [113]:
# Retriever Agent
def take_action(state: AgentState) -> AgentState:
    """Execute tool calls from the LLM's response."""

    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
        print(f"Calling Tool: {t['name']} with query: {t['args'].get('query', 'No query provided')}")
        
        if not t['name'] in tools_dict: # Checks if a valid tool is present
            print(f"\nTool: {t['name']} does not exist.")
            result = "Incorrect Tool Name, Please Retry and Select tool from List of Available tools."
        
        else:
            result = tools_dict[t['name']].invoke(t['args'].get('query', ''))
            print(f"Result length: {len(str(result))}")
            
        # Appends the Tool Message
        results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))

    print("Tools Execution Complete. Back to the model!")
    return {'messages': results}

graph = StateGraph(AgentState)
graph.add_node("retriever_agent", take_action)
graph.add_node("llm", call_llm)

graph.add_conditional_edges(
    "llm",
    should_continue,
    {True: "retriever_agent", False: END}
)
graph.add_edge("retriever_agent", "llm")
graph.set_entry_point("llm")

rag_agent = graph.compile()

In [114]:
def running_agent():
    print("\n=== RAG AGENT===")
    
    while True:
        user_input = input("\nWhat is your question: ")
        if user_input.lower() in ['exit', 'quit']:
            break
            
        messages = [HumanMessage(content=user_input)] # converts back to a HumanMessage type

        result = rag_agent.invoke({"messages": messages})
        
        print("\n=== ANSWER ===")
        print(result['messages'][-1].content)

running_agent()


=== RAG AGENT===
Calling Tool: retriever_tool with query: stock market performance 2024
Result length: 4664
Tools Execution Complete. Back to the model!

=== ANSWER ===
The PDF document provides a detailed overview of the stock market performance in 2024, highlighting several key points:

1. **Overall Market Performance**: The U.S. stock market had a strong year in 2024, with the S&P 500 index delivering a total return of approximately 25%, marking the second consecutive year of over 20% returns. The Nasdaq Composite outperformed the broader market with a nearly 29% increase (Document 1).

2. **Sector Performance**: The technology sector dominated the market, with mega-cap technology stocks leading the rally. A group of companies known as the "Magnificent 7" (Apple, Microsoft, Alphabet, Amazon, Meta, Nvidia, and Tesla) surged by roughly 64-67% on average, contributing significantly to the S&P 500's gains (Document 4).

3. **Market Concentration**: The gains were not evenly distributed